In [ ]:
import time
import sys
print(time.time())
sys.path.append('/home/cjw/Code/Unet')
#sys.path.append('/media/cjw/PythonLib/Unet')
from skimage.io import imread
from skimage.transform import resize
from matplotlib import pyplot as plt
import numpy as np

import unet3d

%matplotlib inline

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
params = dict()

params['width'] = 256
params['height'] = 256
params['depth'] = 8
params['nchannels'] = 2
params['channels'] = [0, 1]
params['nepochs'] = [1]
params['batchsize'] = [16]
params['learning_rate'] = 0.001
params['restore'] = False
params['latent_size'] = 1
params['enc_sizes'] = [[32,3, (2,2,2)], [64, 3, (1,2,2)],
                       [128, 3, (1,2,2)], [256, 3, (1,2,2)]]

params['dec_sizes'] = [[128,3, (1,2,2)], [64, 3, (1,2,2)],
                       [32, 3, (1,2,2)], [1, 3, (2,2,2)]] #[params['nchannels'], 3, (2,2,2)]]
params['droprate'] = 0.1
params['stdev'] = 0.0001

In [ ]:

tf.reset_default_graph()
u = unet3d.unet3d(params)


In [ ]:
#u.read_images('/ssd1/cjw/Data/Nuclei.tif')
t1 = time.time()
datax = np.load('/n/core/micro/asa/fgm/smc/20190919_Screen/cjw_training/augmented_b.npy')
print(time.time() - t1)
datax = datax.astype(np.float32)
#u.read_mm('/n/core/micro/asa/fgm/smc/20190919_Screen/cjw_training/data.mm', shape=(960, 8, 3, 256, 256))

datax.shape, datax.dtype
#u.x = np.moveaxis(u.x, 2, -1)

#u.x.shape, u.x.min(), u.x.max()

In [ ]:
u.x = datax
u.x = np.moveaxis(u.x, 2, -1)
u.x = u.x[np.where(u.x[:,:,:,:,2].max(axis=(1,2,3)) > .05)[0]]
u.x.shape

In [ ]:
validx = np.load('/n/core/micro/asa/fgm/smc/20190919_Screen/cjw_training/validation_b.npy')
validx = np.moveaxis(validx, 2, -1)
validx.shape

In [ ]:
# b = np.reshape(u.x, (720,48, 2, 256, 256, 2))
# b0 = b[:,:,:,:,:,0]
# b1 = b[:,:,:,:,:,1]
# v0 = b0.mean(axis=2)
# v1 = b1.max(axis=2)

print(u.x[66, 5, :,:,2].min())
print(u.x[66, 5, :,:,2].max())
plt.imshow(u.x[66, 5, :,:,2])

In [ ]:
v = u.x #np.stack([v0,v1], axis=-1)
print(v.shape)

In [ ]:
if 1 == 0:
    count = 1
    plt.figure(figsize=(12,48))
    for i in range(16):
        for j in range(6):
            plt.subplot(16,7,count)
            plt.imshow(b[23,i, j, :,:, 0])
            count += 1
        plt.subplot(16,7, count)
        plt.imshow(v[23,i, :,:,0])
        count += 1
#a = u.get_batch(4, augment=True)
#ab = u.augment_batch(a)
#ab.shape

In [ ]:
u.x = v

In [ ]:
# from scipy.ndimage import rotate
# # angle = 90*np.random.randint(0,4)
# # print(angle)
# # s = a[2]
# #ar = rotate(a, 270, (1,2), reshape=False, order=1)
# plt.subplot(2,1,1)
# plt.imshow(a[1,:,:,47,0])
# plt.subplot(2,1,2)
# plt.imshow(a[1,:,:,47,1])
v0 = None
v1 = None
b = None

In [ ]:
tf.reset_default_graph()
images = tf.placeholder(tf.float32, (None, 8, 256, 256, params['nchannels']))
masks = tf.placeholder(tf.float32, (None, 8, 256, 256, 1))
learning_rate = tf.placeholder(tf.float32, ())
u.learning_rate = learning_rate

In [ ]:
enc = u.create_encoder(images, True)

In [ ]:
d = u.create_decoder()
u.create_loss(masks)
u.create_opt()

In [ ]:

u.xtrain = u.x
u.xvalid = validx
#u.set_validation(20)
u.xtrain.shape, u.xvalid.shape

In [ ]:
#u.x[26, 5, :,:,2]

tf.summary.scalar('loss', u.loss)
si = tf.slice(u.decoder, [5, 4, 0, 0, 0], [10,5,256,256, 1])

si = si[:,:,:,:,0]
print(si)
tf.summary.image('res', u.decoder, max_outputs=4)    
merged = tf.summary.merge_all()

sess = tf.Session()

logdir = "/scratch/cjw/fgm/logdir"
logwriter = tf.summary.FileWriter(logdir, sess.graph)

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
u.stdev = 0.02
#u.learning_rate = 0.0001

In [ ]:


vx = u.xvalid[:,:,:,:,0:2]
vm = u.xvalid[:,:,:,:,[2]]

rate = 0.001
#loss_file = open('progress.dat', 'w', buffering=1)
batchsize = 16
for epoch in range(100):
    np.random.shuffle(u.xtrain)
    niter = u.xtrain.shape[0]//batchsize
    #print(niter)
    bindex = 0
    for i in range(niter):
        t1 = time.time()
        rate = np.maximum(rate*.9991, 0.0001)
        batch = u.xtrain[bindex:bindex + batchsize,:,:,:,:]   #u.get_batch(16, augment=False)

        bindex += batchsize
        if bindex > u.xtrain.shape[0]:
            bindex = 0
        bx = batch[:,:,:,:,0:2]
        #bx += np.random.randn()
        bm = batch[:,:,:,:,[2]]
        bm = np.where(bm > .25, 1.0, 0.0)
        #print(bx.shape, bm.shape)
        _, res, err = sess.run([u.opt, u.decoder_sigmoid, u.loss],
                               feed_dict={images:bx, masks:bm, learning_rate:rate})
    if (epoch + 1) % 10 == 0:
        vres, verr = sess.run([u.decoder_sigmoid, u.loss],
                               feed_dict={images:vx, masks:vm})

        zm = vres[0,:,:,:,0].argmax(axis=(-1))
        print(epoch, i, err, verr, vres[0,:,:,:,0].min(), vres[0,:,:,:,0].max())

        nr = 4 #np.random.randint(0,16)
        plt.figure(figsize=(12,2))
        plt.subplot(1,6,1)
        plt.imshow(res[nr,4,:,:,0])
        plt.subplot(1,6,2)
        plt.imshow(bm[nr,4,:,:,0])
        plt.subplot(1,6,3)
        plt.imshow(bx[nr, 4,:,:,0])
        plt.subplot(1,6,4)
        plt.imshow(vres[1, 5,:,:,0])
        plt.subplot(1,6,5)
        plt.imshow(vm[1, 5,:,:,0])
        plt.subplot(1,6,6)
        plt.imshow(vx[1, 5,:,:,0])
        plt.show()
            #plt.savefig("images/image{0:05d}".format(i))
        t2 = time.time()
    print("end of epoch", epoch, err)
#     if i % 20 == 0:
#         print(i, err, (t2 - t1), rate)
# #         loss_file.write("{}, {}, {}, {}\n".format(i, err, (t2 - t1), rate))
#         summary = sess.run(merged,
#                            feed_dict={images:vx, masks:vm})
#         logwriter.add_summary(summary, i)  

#loss_file.close()

In [ ]:
saver = tf.train.Saver()
saver.save(sess, 'fgm/fgm_1.ckpt')

In [ ]:
val_res, val_loss = sess.run([u.decoder_sigmoid, u.loss],
                               feed_dict={images:vx, masks:vm})


In [ ]:
val_res.shape#, valtif.shape

In [ ]:
val_stack = np.concatenate([u.xvalid, val_res], axis=-1)
val_stack.shape

In [ ]:
import tifffile
tifffile.imwrite('/n/core/micro/asa/fgm/smc/20190919_Screen/cjw_training/val_stackb.tif',
                 val_stack.astype(np.float32))

In [ ]:
import tifffile
valtif = tifffile.imread('/n/core/micro/asa/fgm/smc/20190919_Screen/cjw_training/val_patches.tif')
valtif = np.moveaxis(valtif, 2, -1)
valtif.shape
valtif.max()

In [ ]:
val_patches_res = sess.run([u.decoder_sigmoid],
                               feed_dict={images:valtif})

val_cat = np.concatenate([valtif, val_patches_res[0]], axis=-1)
tifffile.imwrite('/n/core/micro/asa/fgm/smc/20190919_Screen/cjw_training/val_patches_res.tif', val_cat)

In [ ]:
print(vx.shape, vm.shape, vres.shape)

xo = np.stack((vx, vm, vres), 4)
xo = np.squeeze(xo)
#xo = np.rollaxis(xo, 3, 1)
xo.shape

In [ ]:
smm = np.memmap('/media/cjw/Data/c3d0.mm', mode='w+', dtype=np.float32
                , shape=xo.shape)

print(type(smm))
smm[:] = xo[:]
print(type(smm))
smm.flush()
del smm

In [ ]:

import tifffile

In [ ]:
tifffile.imwrite('/ssd1/cjw/Data/w3_2-48.tif', xo)

In [ ]:
u.create_loss(masks)
ss = sess.run(u.loss, feed_dict={images:yy, masks:bb})

In [ ]:
ss.shape
plt.imshow(ss[0,:,:,44,0])

In [ ]:
rt = y[0,:,:,:,:]
from scipy.ndimage import rotate

res = rotate(rt, 45, (0, 1), order=2)

In [ ]:
plt.imshow(res[:,:,44,0])

In [ ]:
u.x.shape
u.used = None

In [ ]:
import pickle
from scipy.ndimage import rotate


def augment(b, da = 1):
    mm = np.memmap('/ssd1/cjw/Data/nuclei.mm', dtype=np.float32,
               mode='r+', shape=(720, 256, 256, 96, 2))
    nr = b.shape[0]
    d = dict()
    counter = 0
    for i in range(nr):
        print(i)
        s = b[i]  # this loses the first dimension
        mm[counter] = s
        counter += 1
        print(counter)
        angle = 0
        while angle < 360:
            angle += da
            mm[counter] = rotate(s, angle, (0,1), reshape=False, order=1)
            counter += 1
            print(counter)
            
        mm.flush()
    del mm
        
augment(u.x, 18)

In [ ]:
mm = np.memmap('/media/cjw/Data/nuclei.mm', dtype=np.float32,
               mode='r', shape=(720, 256, 256, 96, 2))

In [ ]:
plt.imshow(mm[-19,:,:,34,0])

In [ ]:
rf = .1
for i in range(20000):
    rf = rf*.9996
    
print(rf)